In [36]:
import numpy as np
import pandas as pd
from scipy.sparse import issparse
from pathlib import Path
import scanpy as sc
import geopandas as gpd

In [44]:
from mygene import MyGeneInfo

ModuleNotFoundError: No module named 'mygene'

In [37]:
# Define path
sample_id = "F07833"
segmentation_path = Path(
    "/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/"
    "dietary_droject/data/cell_segmentation/SegmentedData"
)
adata_path = segmentation_path / sample_id / f"{sample_id}_grouped_filtered_adata.h5ad"

# Load AnnData
ST_sample = sc.read_h5ad(adata_path)

# Load geo data
GDF_file = Path("/mnt/c/Users/jonan/Documents/1Work/RoseLab/Spatial/dietary_droject/data/cell_segmentation/SegmentedData/F07833/F07833_gdf.gpkg")
geo_file = gpd.read_file(GDF_file)

In [3]:
from scanpy.datasets import cell_cycle
s_genes, g2m_genes = cell_cycle()       # these are human symbols

# Upper‑case to match your AnnData
var_up = ST_sample.var_names.str.upper()
s_mouse   = [g.upper() for g in s_genes   if g.upper()   in var_up]
g2m_mouse = [g.upper() for g in g2m_genes if g.upper() in var_up]

print(f"{len(s_mouse)} / {len(s_genes)} S‑genes found in your data")
print(f"{len(g2m_mouse)} / {len(g2m_genes)} G2/M‑genes found in your data")


ImportError: cannot import name 'cell_cycle' from 'scanpy.datasets' (/home/jon/anaconda3/envs/spatial_analysis/lib/python3.10/site-packages/scanpy/datasets/__init__.py)

In [4]:
import scanpy as sc
print(sc.__version__)

1.11.1


In [6]:
sc.tl.score_genes

<function scanpy.tools._score_genes.score_genes(adata: 'AnnData', gene_list: 'Sequence[str] | pd.Index[str]', *, ctrl_as_ref: 'bool' = True, ctrl_size: 'int' = 50, gene_pool: 'Sequence[str] | pd.Index[str] | None' = None, n_bins: 'int' = 25, score_name: 'str' = 'score', random_state: '_LegacyRandom' = 0, copy: 'bool' = False, use_raw: 'bool | None' = None, layer: 'str | None' = None) -> 'AnnData | None'>

In [13]:
# ribo_genes = ST_sample.var_names.str.startswith(('RPS','RPL'))
ribo_genes = ST_sample.var_names.str.upper().str.startswith(('RPS','RPL'))
# ST_sample.var['ribo'] = ribo_genes


# sc.pp.calculate_qc_metrics(
#     ST_sample,
#     qc_vars=['ribo'],
#     inplace=True
# )

In [17]:
ST_sample.var_names

Index(['Xkr4', 'Rp1', 'Sox17', 'Lypla1', 'Tcea1', 'Rgs20', 'Atp6v1h', 'Oprk1',
       'Npbwr1', 'Rb1cc1',
       ...
       'mt-Atp8', 'mt-Atp6', 'mt-Co3', 'mt-Nd3', 'mt-Nd4l', 'mt-Nd4', 'mt-Nd5',
       'mt-Nd6', 'mt-Cytb', 'Vamp7'],
      dtype='object', length=19059)

In [26]:
sum(ribo_genes)

np.int64(0)

In [8]:
ST_sample.var['ribo'].describe()

count     19059
unique        1
top       False
freq      19059
Name: ribo, dtype: object

In [35]:
var_up

Index(['XKR4', 'RP1', 'SOX17', 'LYPLA1', 'TCEA1', 'RGS20', 'ATP6V1H', 'OPRK1',
       'NPBWR1', 'RB1CC1',
       ...
       'MT-ATP8', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4', 'MT-ND5',
       'MT-ND6', 'MT-CYTB', 'VAMP7'],
      dtype='object', length=19059)

In [40]:
ST_sample.shape

(187600, 19059)

In [39]:
# Case-normalize and match
var_up = ST_sample.var_names.str.upper()
ribo_genes = var_up.str.startswith(('RP'))

# Pull the matched gene names (original case)
matched_genes = ST_sample.var_names[ribo_genes]
print(matched_genes)

Index(['Rp1', 'Rpe', 'Rpp38', 'Rprm', 'Rpusd2', 'Rpap1', 'Rpn2', 'Rprd1b',
       'Rptn', 'Rprd2', 'Rpf1', 'Rpe65', 'Rpp25l', 'Rpa2', 'Rpap2', 'Rph3a',
       'Rpa3', 'Rpia', 'Rpn1', 'Rpusd3', 'Rpgrip1l', 'Rp9', 'Rpusd4', 'Rpp25',
       'Rpf2', 'Rpain', 'Rpa1', 'Rph3al', 'Rprml', 'Rptor', 'Rpp40', 'Rpp14',
       'Rp1l1', 'Rpap3', 'Rpusd1', 'Rpp21', 'Rprd1a', 'Rpp30', 'Rpgr', 'Rp2'],
      dtype='object')


In [38]:
var_up = ST_sample.var_names.str.upper()
ribo_genes = var_up.str.startswith(('RPS','RPL'))
print(f"{ribo_genes.sum()} ribosomal genes flagged")
# ST_sample.var['ribo'] = ribo_genes

0 ribosomal genes flagged


In [31]:
ribo_genes

array([False,  True, False, ..., False, False, False], shape=(19059,))

In [19]:
ribo_genes

array([False, False, False, ..., False, False, False], shape=(19059,))

In [27]:
var_up

Index(['XKR4', 'RP1', 'SOX17', 'LYPLA1', 'TCEA1', 'RGS20', 'ATP6V1H', 'OPRK1',
       'NPBWR1', 'RB1CC1',
       ...
       'MT-ATP8', 'MT-ATP6', 'MT-CO3', 'MT-ND3', 'MT-ND4L', 'MT-ND4', 'MT-ND5',
       'MT-ND6', 'MT-CYTB', 'VAMP7'],
      dtype='object', length=19059)

In [41]:
[r for r in ['Rpl13a', 'Rps3', 'Rplp1', 'Rps6'] if r in ST_sample.var_names]


[]

In [42]:
any(ST_sample.var_names.str.upper().str.startswith('RPL'))

False

In [43]:
any(ST_sample.var_names.str.upper().str.startswith('RPS'))

False